In [1]:
import os

os.environ['NEPTUNE_PROJECT'] = 'ling583/ling583-anish'
os.environ[
    'NEPTUNE_API_TOKEN'] = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxNzU4ZjdlZi1lZjhkLTQxZmItOGYzOC03ZThkMDVlMDVmZWUifQ=='


In [2]:
import numpy as np
import pandas as pd
import neptune
from neptune.utils import stringify_unsupported

from sklearn.metrics import accuracy_score, classification_report, f1_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    TextClassificationPipeline,
)
from datasets import load_from_disk
from transformers.pipelines.pt_utils import KeyDataset

from tqdm.auto import tqdm

pd.options.mode.copy_on_write = True
pd.options.display.max_colwidth = 500

/Users/anishchintamaneni/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MAX_LEN = 512
TOKENIZER_ARGS = {
    "padding": "max_length",
    "truncation": True,
    "max_length": MAX_LEN,
}


def tokenize_function(item):
    return tokenizer(item["text"], return_tensors="np", **TOKENIZER_ARGS)

In [4]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1_score": f1_score(labels, predictions, average="macro"),
    }

In [5]:
dataset = load_from_disk("lab08_dataset")

In [6]:
MODEL = "distilbert/distilbert-base-uncased"
BATCH_SIZE = 32

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4)

In [8]:
args = TrainingArguments(
    output_dir="runs",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_steps=10,
    tf32=True,
    bf16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1_score",
    save_total_limit=1,
    num_train_epochs=12,
    torch_compile=False,
    gradient_accumulation_steps=1,
    report_to="none",
)

args = args.set_dataloader(
    train_batch_size=BATCH_SIZE, eval_batch_size=BATCH_SIZE, num_workers=1
)

ValueError: BF16 Mixed precision training with AMP (`--bf16`) and BF16 half precision evaluation (`--bf16_full_eval`) can only be used on CUDA, XPU (with IPEX), NPU or CPU/TPU/NeuronCore devices.

In [12]:
pip install transformers[torch]

zsh:1: no matches found: transformers[torch]
Note: you may need to restart the kernel to use updated packages.
